In [1]:
%load_ext pycodestyle_magic
%pycodestyle_on
# %pycodestyle_off -to turn it off

In [35]:
# import all libraries
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_timestamp
import requests
import json
import pandas as pd
from binance import Client
from datetime import datetime
import os
from dotenv import load_dotenv, dotenv_values

load_dotenv()

True

In [3]:
API_KEY = os.getenv("API_KEY")
SECRET_KEY = os.getenv("SECRET_KEY")
client = Client(API_KEY, SECRET_KEY)
spark = SparkSession.builder.appName("test").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/04 18:31:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/02/04 18:31:50 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
# tickers = client.get_all_tickers()
# client.get_exchange_info() #has rate limit info
# client.response.headers #has info on current usage

1:1: E265 block comment should start with '# '
1:36: W291 trailing whitespace
2:1: E265 block comment should start with '# '
3:1: E265 block comment should start with '# '


In [123]:
# start_datetime = datetime.strptime("01-01-2023", '%d-%m-%Y')
# start_date = start_datetime.strftime("%Y-%m-%dT%H:%M:%SZ")

# end_datetime = datetime.strptime("01-01-2024", '%d-%m-%Y')
# end_date_date = end_datetime.strftime("%Y-%m-%dT%H:%M:%SZ")

In [14]:
start_date = "01-01-2023"
end_date = "12-31-2023"
symbol = "BTCUSDT"
interval = "1M"

In [15]:
klines = client.get_historical_klines(
    symbol=symbol, interval=interval, start_str=start_date, end_str=end_date, limit=1000
)

2:80: E501 line too long (88 > 79 characters)


In [6]:
# api response structure
"""[
  [
    1499040000000,      // Kline open time
    "0.01634790",       // Open price
    "0.80000000",       // High price
    "0.01575800",       // Low price
    "0.01577100",       // Close price
    "148976.11427815",  // Volume
    1499644799999,      // Kline Close time
    "2434.19055334",    // Quote asset volume
    308,                // Number of trades
    "1756.87402397",    // Taker buy base asset volume
    "28.46694368",      // Taker buy quote asset volume
    "0"                 // Unused field, ignore.
  ]
]"""

'[\n  [\n    1499040000000,      // Kline open time\n    "0.01634790",       // Open price\n    "0.80000000",       // High price\n    "0.01575800",       // Low price\n    "0.01577100",       // Close price\n    "148976.11427815",  // Volume\n    1499644799999,      // Kline Close time\n    "2434.19055334",    // Quote asset volume\n    308,                // Number of trades\n    "1756.87402397",    // Taker buy base asset volume\n    "28.46694368",      // Taker buy quote asset volume\n    "0"                 // Unused field, ignore.\n  ]\n]'

In [50]:
columns = [
    "Kline open time",
    "Open Price",
    "High price",
    "Low price",
    "Close Price",
    "Volume",
    "Kline Close time",
    "Quote asset volume",
    "Number of trades",
    "Taker buy base asset volume",
    "Taker buy quote asset volume",
    "Ignore",
]

In [51]:
dataframe = spark.createDataFrame(klines, columns)

In [52]:
df = dataframe.withColumn("Kline open time", to_timestamp(col("Kline open time") / 1000))
df = df.withColumnRenamed("Kline open time","datetime")
df = df.select('datetime','Open Price','Close Price','Volume')

1:80: E501 line too long (89 > 79 characters)
2:44: E231 missing whitespace after ','
3:26: E231 missing whitespace after ','
3:39: E231 missing whitespace after ','
3:53: E231 missing whitespace after ','


In [53]:
df.show()

+-------------------+--------------+--------------+----------------+
|           datetime|    Open Price|   Close Price|          Volume|
+-------------------+--------------+--------------+----------------+
|2023-01-01 00:00:00|16541.77000000|23125.13000000|7977028.87801000|
|2023-02-01 00:00:00|23125.13000000|23141.57000000|8642691.27165000|
|2023-03-01 00:00:00|23141.57000000|28465.36000000|9516189.35846000|
|2023-04-01 01:00:00|28465.36000000|29233.21000000|1626745.55850000|
|2023-05-01 01:00:00|29233.20000000|27210.35000000|1302000.49221000|
|2023-06-01 01:00:00|27210.36000000|30472.00000000|1387207.48275000|
|2023-07-01 01:00:00|30471.99000000|29232.25000000| 925773.81731000|
|2023-08-01 01:00:00|29232.26000000|25940.78000000|1025866.55023000|
|2023-09-01 01:00:00|25940.77000000|26962.56000000| 809329.04893000|
|2023-10-01 01:00:00|26962.57000000|34639.77000000|1141403.67990000|
|2023-11-01 00:00:00|34639.78000000|37723.96000000|1055690.59638000|
|2023-12-01 00:00:00|37723.9700000